In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import dplm_base

%matplotlib inline

In [ ]:
del dplm_instance

In [ ]:
dplm_instance = dplm_base.dplm('para1.csv')
dplm_instance.show_dplm_config()
dplm_instance.set_dplm_slot_num(10)
dplm_instance.set_dplm_spring_num(3)
# dplm_instance.set_springs_positions([0.16,0.16,0.16])
dplm_instance.set_dplm_spring_constants([4.00651466e-02,3.99968180e-01,4.03535235e-02])
dplm_instance.set_dplm_spring_lengths([1.00001904e-01,1.00019105e-01,1.00199074e-01])
dplm_instance.set_springs_positions([3.99868244e+02, 3.30538434e+02,
 3.99935009e+02])
dplm_instance.set_dplm_allowed_angle_range(-20, 60, 1)
# dplm_instance.set_slot([-6, 18, 0])
# moment_weight, moment_spring_list, moment_total = dplm_instance.calculate_current_moment()

# 26 32 38
#13 37 38

In [ ]:
lower_limit, upper_limit, step_size, total_angle_num = dplm_instance.get_allowed_angle_range().values()

a,b,c, rmse = dplm_instance.calculate_current_moment()
%matplotlib inline
plt.cla()    
plt.plot(range(lower_limit, upper_limit+1), a, label = 'moment_weight', ls = '--', lw = 3, color = 'mediumaquamarine')

ax = plt.gca()

for i in range(len(b)):
    plt.plot(range(lower_limit,upper_limit+1), b[i], label = 'moment_spring_{}'.format(i+1), ls = '--', lw = 3, color = 'cornflowerblue')

plt.plot(range(lower_limit, upper_limit+1), c, label = 'moment_total', ls = '--', lw = 3, color = 'mediumslateblue')
plt.axhline(y = 0, ls = '-', lw = 3, color = 'darkgrey')

plt.axis(ymin=-20, ymax=50)
plt.legend()
plt.xlabel('angle [degree]')
plt.ylabel('moment [Nm]')
ax.xaxis.set_major_formatter('{x}°')


plt.text(-10,-10, r'$RMSE={:.2f}$'.format(rmse))


plt.show()

print(rmse)

In [ ]:
import gym
import math
import os
import time
import matplotlib.pyplot as plt
from stable_baselines3.common.env_checker import check_env

# %matplotlib widget

cwd = os.getcwd()
angle_range = {
    'lower_limit' : -20,
    'upper_limit' : 60,
    'step_size' : 1
}

env = gym.make('gym_dplm:dplm-v0', 
                dplm_config_file = cwd+"/para1.csv",
                spring_num = 3,
                slot_num = 20,
                spring_constants = [300,300,300],
                spring_init_lengths = [0.16,0.16,0.16],
                **angle_range)

check_env(env)

## A callback for displaying a progress bar while training

In [ ]:
from tqdm.auto import tqdm
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.results_plotter import load_results, ts2xy

class ProgressBarCallback(BaseCallback):
    """
    :param pbar: (tqdm.pbar) Progress bar object
    """
    def __init__(self, pbar):
        super(ProgressBarCallback, self).__init__()
        self._pbar = pbar

    def _on_step(self):
        # Update the progress bar:
        self._pbar.n = self.num_timesteps
        self._pbar.update(0)

# this callback uses the 'with' block, allowing for correct initialisation and destruction
class ProgressBarManager(object):
    def __init__(self, total_timesteps): # init object with total timesteps
        self.pbar = None
        self.total_timesteps = total_timesteps
        
    def __enter__(self): # create the progress bar and callback, return the callback
        self.pbar = tqdm(total=self.total_timesteps)
            
        return ProgressBarCallback(self.pbar)

    def __exit__(self, exc_type, exc_val, exc_tb): # close the callback
        self.pbar.n = self.total_timesteps
        self.pbar.update(0)
        self.pbar.close()
        

In [ ]:
import gym
import math
import os
import time

from stable_baselines3 import DQN, A2C, PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.cmd_util import make_vec_env

cwd = os.getcwd()
angle_range = {
    'lower_limit' : -20,
    'upper_limit' : 60,
    'step_size' : 1
}

env = gym.make('gym_dplm:dplm-v0', 
                dplm_config_file = cwd+"/para1.csv",
                spring_num = 3,
                slot_num = 20,
                spring_constants = [300,300,300],
                spring_init_lengths = [0.16,0.16,0.16],
                rmse_limit = 3,
                **angle_range)

# env = make_vec_env(lambda: env, n_envs=40)

env = make_vec_env(lambda: env, n_envs=10)
model = PPO('MlpPolicy', env, verbose=1)

model.learn(10000)


# model = A2C('MlpPolicy', env, verbose=1).learn(50000)
# model.save('dplm')

In [ ]:
# Test the trained agent
for i in range(10):
    print('Test: NO.{}'.format(i+1))
    obs = env.reset()
    n_steps = 50
    for step in range(n_steps):
        action, _ = model.predict(obs, deterministic=True)
        print("Step {}".format(step + 1))
        print("Action: ", action)
        obs, reward, done, info = env.step(action)
        print('obs=', obs, 'reward=', reward, 'done=', done)
        # env.render()
        if done:
            # Note that the VecEnv resets automatically
            # when a done signal is encountered
            print("Goal reached!", "reward=", reward)
            break

## Checking if nvidia gpu (cuda) is available

In [ ]:
import torch 
from torch import nn 


print(torch.cuda.is_available()) # true 查看GPU是否可用

print(torch.cuda.device_count()) #GPU数量， 1

torch.cuda.current_device() #当前GPU的索引， 0

torch.cuda.get_device_name(0) #输出GPU名称

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import dplm_base

dplm_instance = dplm_base.dplm('para2.csv')
dplm_instance.show_dplm_config()
dplm_instance.set_dplm_allowed_angle_range(-20, 60, 1)

#How many sets of the same triangle
spring_constant = 52/2
spring_initial_length = 0.184
slot_num = 39

dplm_instance.add_triangle(spring_constant*10, 0.184)
dplm_instance.set_dplm_slot_num(39)
dplm_instance.set_slot([10,1])

# dplm_instance.set_dplm_slot_num(10)

# dplm_instance.set_springs_positions([0.16,0.16,0.16])
# dplm_instance.set_dplm_spring_num(3)
# dplm_instance.set_dplm_spring_constants([4.00651466e-02,3.99968180e-01,4.03535235e-02])
# dplm_instance.set_dplm_spring_lengths([1.00001904e-01,1.00019105e-01,1.00199074e-01])
# dplm_instance.set_springs_positions([3.99868244e+02, 3.30538434e+02,
#  3.99935009e+02])

# dplm_instance.set_slot([-6, 18, 0])
# moment_weight, moment_spring_list, moment_total = dplm_instance.calculate_current_moment()

# 26 32 38

In [ ]:
lower_limit, upper_limit, step_size, total_angle_num = dplm_instance.get_allowed_angle_range().values()

a,b,c, rmse = dplm_instance.calculate_current_moment()
%matplotlib inline
plt.cla()    
plt.plot(range(lower_limit, upper_limit+1), a, label = 'moment_weight', ls = '--', lw = 3, color = 'mediumaquamarine')

ax = plt.gca()

for i in range(len(b)):
    plt.plot(range(lower_limit,upper_limit+1), b[i], label = 'moment_spring_{}'.format(i+1), ls = '--', lw = 3, color = 'cornflowerblue')

plt.plot(range(lower_limit, upper_limit+1), c, label = 'moment_total', ls = '--', lw = 3, color = 'mediumslateblue')
plt.axhline(y = 0, ls = '-', lw = 3, color = 'darkgrey')

plt.axis(ymin=-20, ymax=20)
plt.legend()
plt.xlabel('angle [degree]')
plt.ylabel('moment [Nm]')
ax.xaxis.set_major_formatter('{x}°')


plt.text(-10,-10, r'$RMSE={:.2f}$'.format(rmse))


plt.show()

print(rmse)

In [ ]:
import dplm_base
import numpy as np 
import matplotlib.pyplot as plt
import csv
%matplotlib inline


plotting = True

dplm_instance = dplm_base.dplm('para1.csv')
dplm_instance.show_dplm_config()
dplm_instance.set_dplm_slot_num(20)
dplm_instance.set_dplm_spring_num(4)
# dplm_instance.set_slot([-4, 13, 8])
dplm_instance.set_dplm_spring_constants([600,300,250, 230])
dplm_instance.set_dplm_spring_lengths([0.1, 0.2, 0.17, .13])
dplm_instance.set_dplm_allowed_angle_range(-40, 60, 1)


with open('greedy_graphs/4_springs/greedy.csv', mode='w+', newline='') as csvfile:
    csvfile.writelines('number of springs: {}\n'.format(dplm_instance.get_spring_num()))
    csvfile.writelines('Number of slots: {}\n'.format(dplm_instance.get_slot_num()))
    csvfile.writelines('spring_constants: {}\n'.format(dplm_instance.get_spring_constatnts()))
    csvfile.writelines('spring_lengths: {}\n'.format(dplm_instance.get_spring_init_lengths()))
    
    writer = csv.writer(csvfile, delimiter = ',', quotechar = '"')
    # writer.writerow('"number of springs: {}"'.format(dplm_instance.get_spring_num()))
    # writer.writerow('"Number of slots: {}"'.format(dplm_instance.get_slot_num()))
    # writer.writerow('"spring_constants: {}"'.format(dplm_instance.get_spring_constatnts()))
    # writer.writerow('"spring_lengths: {}"'.format(dplm_instance.get_spring_init_lengths()))
    writer.writerow(['rmse', 'initial guess', 'final install positions'])


    for sample in range(100): #try 100 different initial states
        init_guess = np.random.randint(0, high=dplm_instance.get_slot_num()*2-1, size=dplm_instance.get_spring_num())-dplm_instance.get_slot_num()+1
        guess = np.array(init_guess, copy=True)
        for greedy_iter_num in range(1): #three iterations for each greedy
            for ind in range(dplm_instance.get_spring_num()):
                rmse = np.zeros(dplm_instance.get_slot_num())
                for slot in range(-dplm_instance.get_slot_num()+1, dplm_instance.get_slot_num()):
                    guess[ind] = slot
                    dplm_instance.set_slot(guess)
                    rmse[slot] = dplm_instance.current_rmse()
                    guess[ind] = np.argmin(rmse)
            dplm_instance.set_slot(guess)

        if plotting == True:
            fig = plt.figure(figsize=[9.6, 6.4])
            lower_limit, upper_limit, step_size, total_angle_num = dplm_instance.get_allowed_angle_range().values()

            a,b,c, rmse = dplm_instance.calculate_current_moment()
            plt.cla()    
            # plt.figure()
            ax = plt.gca()
            plt.plot(range(lower_limit, upper_limit+1), a, label = 'moment_weight', ls = '--', lw = 1, color = 'grey')

            for i in range(len(b)):
                plt.plot(range(lower_limit,upper_limit+1), b[i], label = 'moment_spring_{}'.format(i+1), ls = '-', lw = 1, color = 'cornflowerblue')

            plt.plot(range(lower_limit, upper_limit+1), c, label = 'moment_total', ls = '-', lw =4, color = 'gold')
            plt.axhline(y = 0, ls = '-', lw = 3, color = 'darkgrey')

            plt.axis(ymin=-20, ymax=50)
            plt.legend()
            plt.xlabel('angle [degree]')
            plt.ylabel('moment [Nm]')
            ax.xaxis.set_major_formatter("{x}°")


            plt.text(-20,-13, 'RMSE={:.2f} \nInitial random state: {} \nFinal install positions: {}'.format(rmse,init_guess, guess))
            plt.savefig('greedy_graphs/4_springs/test_{}.png'.format(sample+1))
            plt.show()
            # del fig
            
            writer.writerow([f'{rmse:.2f}', list(init_guess), list(guess)])

            print(rmse)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import dplm_base
k = pd.read_csv('greedy_graphs/4_springs_all_para/greedy.csv', header = 4)

k.index = np.arange(1, len(k)+1)
k.index.name='sample'
k.to_excel('greedy_graphs/4_springs_all_para/greedy.xlsx')
k.to_csv('greedy_graphs/4_springs_all_para/greedy_processed.csv')
k.to_clipboard(sep = ',')

k

In [10]:
#Extract the list of action 
import csv

with open("a2c_output.txt","r") as fi:
    id = []
    for ln in fi:
        if ln.startswith("Action:"):
            id.append(ln[8:-1])
with open("a2c_output_processed.csv", 'w+', newline = '') as csvfile:
    csvfile.writelines('action\n')
    writer = csv.writer(csvfile, delimiter=',', quotechar = '"')
    for k in id:
        writer.writerow([k])
    

print(id)

[' [17 16 16 17 17]', ' [17 17 17 17 17]', ' [17 17 17 17 17]', ' [17 17 17 17 17]', ' [17 17 17 17 17]', ' [17 17 16 17 17]', ' [17 17 17 17 17]', ' [17 17 17 16 16]', ' [16 16 16 16 16]', ' [16 16 17 17 17]', ' [17 17 17 17 17]', ' [17 16 16 16 16]', ' [16 16 16 16 16]', ' [17 17 17 17 17]', ' [17 17 17 17 16]', ' [16 16 16 16 16]', ' [16 16 16 17 17]', ' [17 17 17 17 17]', ' [17 17 16 16 16]', ' [16 16 16 16 16]', ' [16 17 17 17 17]', ' [17 17 17 17 17]', ' [16 16 16 16 16]', ' [16 16 16 16 17]', ' [17 17 17 17 17]', ' [17 17 17 16 16]', ' [16 16 16 16 16]', ' [16 16 17 17 17]', ' [17 17 17 17 17]', ' [17 16 16 16 16]', ' [16 16 16 16 16]', ' [17 17 17 17 17]', ' [17 17 17 17 16]', ' [16 16 16 16 16]', ' [16 16 16 17 17]', ' [17 17 17 17 17]', ' [17 17 16 16 16]', ' [16 16 16 16 16]', ' [16 17 17 17 17]', ' [17 17 17 17 17]', ' [16 16 16 16 16]', ' [16 16 16 16 17]', ' [17 17 17 17 17]', ' [17 17 17 16 16]', ' [16 16 16 16 16]', ' [16 16 17 17 17]', ' [17 17 17 17 17]', ' [17 16 16 

In [56]:
#DQN training results

%gui osx
%matplotlib

import pandas as pd
import matplotlib.pyplot as plt

files = ['./dplm_tensorboards/curves/run-DQN_1-tag-rollout_ep_len_mean.csv',
         './dplm_tensorboards/curves/run-DQN_1-tag-rollout_ep_rew_mean.csv']

titles = ['Mean episode length', 'Mean episode reward']

figure, axes = plt.subplots(1, 2,figsize=[9.6, 4.8], dpi=100)

# ep_mean_len = files[0]
# ep_mean_rew = files[1]
# print(k.head)
plt.suptitle('DQN Training Results', size = 15, weight = 'semibold')

plt.subplot(1,2,1)
axes[0].set_title(titles[0], color = 'maroon', weight = 'semibold')
axes[0].set_xlabel('Step')
axes[0].set_ylabel('Mean length')
k = pd.read_csv(files[0], usecols=[1,2], index_col=0)
print(k)
x = k.index
y = k['Value']
axes[0].plot(x,y)

plt.subplot(1,2,2)
axes[1].set_title(titles[1], color = 'maroon', weight = 'semibold')
axes[1].set_xlabel('Step')
axes[i].set_ylabel('Mean reward')
k = pd.read_csv(files[1], usecols=[1,2], index_col=0)
plt.subplot(1,2,1)
x = k.index
y = k['Value']
axes[1].plot(x,y)

# ep_mean_rew = pd.read_csv()

Using matplotlib backend: MacOSX
        Value
Step         
122   30.5000
154   19.2500
204   17.0000
263   16.4375
319   15.9500
...       ...
9765  16.9800
9807  16.4000
9837  15.6800
9919  15.8400
9946  15.9600

[164 rows x 1 columns]


In [70]:
#A2C training results

%gui osx
%matplotlib

import pandas as pd
import matplotlib.pyplot as plt

files = ['./dplm_tensorboards/curves/run-A2C_3-tag-rollout_ep_len_mean.csv',
         './dplm_tensorboards/curves/run-A2C_6-tag-rollout_ep_len_mean.csv',
         './dplm_tensorboards/curves/run-A2C_3-tag-rollout_ep_rew_mean.csv',
         './dplm_tensorboards/curves/run-A2C_6-tag-rollout_ep_rew_mean.csv'
        ]

titles = ['Mean episode length', 'Mean episode reward']

figure, axes = plt.subplots(1, 2,figsize=[9.6, 4.8], dpi=100)

# ep_mean_len = files[0]
# ep_mean_rew = files[1]
# print(k.head)
plt.suptitle('A2C Training Results', size = 15, weight = 'semibold')

plt.subplot(1,2,1)
axes[0].set_title(titles[0], color = 'maroon', weight = 'semibold')
axes[0].set_xlabel('Step')
axes[0].set_ylabel('Mean length')
k = pd.read_csv(files[0], usecols=[1,2], index_col=0)
k_2 = pd.read_csv(files[1], usecols=[1,2], index_col=0)
print(k)
x = k.index
y = k['Value']
x_2 = k_2.index
y_2 = k_2['Value']
axes[0].plot(x_2,y_2, label='100000 steps',linewidth=3)
axes[0].plot(x,y,label='30000 steps',linewidth=3)
axes[0].legend()

plt.subplot(1,2,2)
axes[1].set_title(titles[1], color = 'maroon', weight = 'semibold')
axes[1].set_xlabel('Step')
axes[i].set_ylabel('Mean reward')
k = pd.read_csv(files[2], usecols=[1,2], index_col=0)
k_2 = pd.read_csv(files[3], usecols=[1,2], index_col=0)
# plt.subplot(1,2,1)
x = k.index
y = k['Value']
x_2 = k_2.index
y_2 = k_2['Value']
axes[1].plot(x_2,y_2, label='100000 steps',linewidth=3)
axes[1].plot(x,y,label='30000 steps',linewidth=3)
axes[1].legend()

Using matplotlib backend: MacOSX
           Value
Step            
2500   15.690000
5000   16.340000
7500   16.700001
10000  17.139999
12500  15.150000
15000  22.590000
17500  28.750000
20000  35.330002
22500  48.709999
25000  62.540001
27500  74.870003
30000  77.610001
32500  96.970001


In [78]:
#A2C losses

%gui osx
%matplotlib

import pandas as pd
import matplotlib.pyplot as plt

files = ['./dplm_tensorboards/curves/run-A2C_3-tag-train_entropy_loss.csv',
         './dplm_tensorboards/curves/run-A2C_6-tag-train_entropy_loss.csv',
         './dplm_tensorboards/curves/run-A2C_3-tag-train_policy_loss.csv',
         './dplm_tensorboards/curves/run-A2C_6-tag-train_policy_loss.csv',
         './dplm_tensorboards/curves/run-A2C_3-tag-train_value_loss.csv',
         './dplm_tensorboards/curves/run-A2C_6-tag-train_value_loss.csv'
        ]

# run-A2C_6-tag-train_policy_loss.csv
# run-A2C_3-tag-train_entropy_loss.csv       run-A2C_6-tag-train_value_loss.csv
# run-A2C_3-tag-train_policy_loss.csv        run-A2C_6-tag-train_entropy_loss.csv
# run-A2C_3-tag-train_value_loss.csv  

titles = ['Entropy loss', 'Policy loss', 'Value loss']

figure, axes = plt.subplots(2, 2,figsize=[9.6, 7.2], dpi=100)
figure.tight_layout(pad = 3)
plt.subplots_adjust(left=0.08, bottom=.08, right=.96, top=.9)
# ep_mean_len = files[0]
# ep_mean_rew = files[1]
# print(k.head)
plt.suptitle('A2C Training losses', size = 15, weight = 'semibold')

plt.subplot(2,2,1)
axes[0][0].set_title(titles[0], color = 'maroon', weight = 'semibold')
axes[0][0].set_xlabel('Step')
axes[0][0].set_ylabel('Entropy loss')
k = pd.read_csv(files[0], usecols=[1,2], index_col=0)
k_2 = pd.read_csv(files[1], usecols=[1,2], index_col=0)
print(k)
x = k.index
y = k['Value']
x_2 = k_2.index
y_2 = k_2['Value']
axes[0][0].plot(x_2,y_2, label='100000 steps',linewidth=3)
axes[0][0].plot(x,y,label='30000 steps',linewidth=3)
axes[0][0].legend()

plt.subplot(2,2,2)
axes[0][1].set_title(titles[1], color = 'maroon', weight = 'semibold')
axes[0][1].set_xlabel('Step')
axes[0][1].set_ylabel('Policy loss')
k = pd.read_csv(files[2], usecols=[1,2], index_col=0)
k_2 = pd.read_csv(files[3], usecols=[1,2], index_col=0)
# plt.subplot(1,2,1)
x = k.index
y = k['Value']
x_2 = k_2.index
y_2 = k_2['Value']
axes[0][1].plot(x_2,y_2, label='100000 steps',linewidth=3)
axes[0][1].plot(x,y,label='30000 steps',linewidth=3)
axes[0][1].legend()


plt.subplot(2,2,3)
axes[1][0].set_title(titles[2], color = 'maroon', weight = 'semibold')
axes[1][0].set_xlabel('Step')
axes[1][0].set_ylabel('Policy loss')
k = pd.read_csv(files[4], usecols=[1,2], index_col=0)
k_2 = pd.read_csv(files[5], usecols=[1,2], index_col=0)
# plt.subplot(1,2,1)
x = k.index
y = k['Value']
x_2 = k_2.index
y_2 = k_2['Value']
axes[1][0].plot(x_2,y_2, label='100000 steps',linewidth=3)
axes[1][0].plot(x,y,label='30000 steps',linewidth=3)
axes[1][0].legend()

Using matplotlib backend: MacOSX
          Value
Step           
2500  -3.175370
5000  -3.172108
7500  -3.165416
10000 -3.145498
12500 -3.126778
15000 -3.035429
17500 -2.849458
20000 -2.057361
22500 -1.874383
25000 -1.336397
27500 -1.288422
30000 -1.196335
32500 -1.001392
